## 주식 가격 수집 및 저장 (대성에너지)
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장
+ 작성: 임경호

#### 0. 데이터를 가져올 날짜 설정

In [1]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

stock_code = '117580'   # 대성에너지 stock code

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT tr_date FROM stock_price where stock_code = {stock_code} ORDER BY tr_date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

start_date = myutil.get_next_day(last_date)
end_date = myutil.get_previous_day()

In [2]:
stock_code, last_date, start_date, end_date

('117580', '20230726', '20230727', '20230903')

#### 1. 주가 데이터 가져오기

In [3]:
from pykrx import stock

# 특정기간 특정 종목의 OHLCV
ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)

In [4]:
ohlcv.shape

(26, 7)

In [5]:
ohlcv

,시가,고가,저가,종가,거래량,거래대금,등락률
날짜,,,,,,,
2023-07-27,6760,7040,6760,6900,67831,470806110,1.77
2023-07-28,6820,7170,6820,7110,59605,419883110,3.04
2023-07-31,7060,7240,7060,7200,37336,267322440,1.27
2023-08-01,7200,7360,7190,7300,59431,430741060,1.39
2023-08-02,7260,7310,7130,7210,55926,404106540,-1.23
2023-08-03,7070,7180,7060,7170,44994,320573150,-0.55
2023-08-04,7170,7330,7090,7250,41855,301864020,1.12
2023-08-07,7260,7290,7160,7200,65218,471555160,-0.69
2023-08-08,7360,7360,7200,7210,53713,389741870,0.14


#### 2. 데이터 변환 및 전처리

In [6]:
df_ohlcv = ohlcv.reset_index()
df_ohlcv

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2023-07-27,6760,7040,6760,6900,67831,470806110,1.77
1,2023-07-28,6820,7170,6820,7110,59605,419883110,3.04
2,2023-07-31,7060,7240,7060,7200,37336,267322440,1.27
3,2023-08-01,7200,7360,7190,7300,59431,430741060,1.39
4,2023-08-02,7260,7310,7130,7210,55926,404106540,-1.23
5,2023-08-03,7070,7180,7060,7170,44994,320573150,-0.55
6,2023-08-04,7170,7330,7090,7250,41855,301864020,1.12
7,2023-08-07,7260,7290,7160,7200,65218,471555160,-0.69
8,2023-08-08,7360,7360,7200,7210,53713,389741870,0.14
9,2023-08-09,7200,7340,7140,7330,37903,274654360,1.66


In [7]:
# 날짜 형식 변환 => 문자열 yyyymmdd
df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')

In [8]:
df_ohlcv

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,20230727,6760,7040,6760,6900,67831,470806110,1.77
1,20230728,6820,7170,6820,7110,59605,419883110,3.04
2,20230731,7060,7240,7060,7200,37336,267322440,1.27
3,20230801,7200,7360,7190,7300,59431,430741060,1.39
4,20230802,7260,7310,7130,7210,55926,404106540,-1.23
5,20230803,7070,7180,7060,7170,44994,320573150,-0.55
6,20230804,7170,7330,7090,7250,41855,301864020,1.12
7,20230807,7260,7290,7160,7200,65218,471555160,-0.69
8,20230808,7360,7360,7200,7210,53713,389741870,0.14
9,20230809,7200,7340,7140,7330,37903,274654360,1.66


#### 3. DB 테이블에 저장

In [9]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_ohlcv.itertuples():
    sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                            values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
conn.commit()
conn.close()